## We are trying to win the wrong game.

In [1]:
def x_sq(x):
    return x**2

def x2(x):
    return 2*x

In [2]:
def num_grad(f, x, h=1e-8):
    return (f(x+h) - f(x)) / h

In [3]:
x2(1), num_grad(x_sq, 0.001)

(2, 0.002000009999999128)

In [4]:
1e-2

0.01

In [5]:
import fromscratchtoml
fromscratchtoml.use("numpy")
# fromscratchtoml.use("cupy")

from fromscratchtoml.neural_network.models import Sequential
from fromscratchtoml.neural_network.optimizers import StochasticGradientDescent
from fromscratchtoml.neural_network.layers import Dense, Activation, Dropout
from fromscratchtoml.neural_network.regularizers import l1, l2, l1_l2

import numpy as np
# import cupy as np

from sklearn.model_selection import train_test_split

from fromscratchtoml.toolbox.preprocess import to_onehot

from sklearn import datasets
from sklearn.utils import shuffle

from fromscratchtoml.neural_network.losses import cross_entropy
from copy import deepcopy

%matplotlib inline

In [6]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home=".")
print(len(mnist.data))
X2 = mnist.data/256.
mnist.target.__class__
y2 = to_onehot(mnist.target)

70000


In [7]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=10000, random_state=42)
X_train2.shape, y_train2.shape, X_train2.dtype

((60000, 784), (60000, 10), dtype('float64'))

In [8]:
# print(X_train2.shape)
# X_train2 = np.expand_dims(X_train2[0], axis=1).T

# y_train2 = np.expand_dims(y_train2[0], axis=1).T
# X_train2.shape, y_train2.shape

In [9]:
## START

In [10]:
# get w
layer = Dense(10, input_dim=784, seed=1)
y = layer.forward(deepcopy(X_train2), train=True)
w = layer.weights

E = cross_entropy(deepcopy(y), deepcopy(y_train2))

In [11]:
w

array([[ 0.65387455, -0.92132293, -0.14418936, ..., -0.39259022,
         0.01887856,  0.26064289],
       [-0.49221186,  0.51193601,  0.40320363, ..., -0.41848881,
        -0.11980319,  0.23718218],
       [-0.30932009, -0.17743357, -0.30731297, ...,  0.10483389,
         0.7422861 ,  0.33185224],
       ...,
       [ 0.48671913, -0.08406576, -0.44510272, ...,  0.00370278,
        -0.63047716, -0.40293975],
       [ 0.29035899, -0.39472819,  0.39954523, ...,  0.28816253,
        -0.68040612, -0.11703643],
       [-0.44562584, -0.48717856,  0.05977087, ...,  0.37613845,
        -0.15007702,  0.21746986]])

In [12]:
# get w+
h = 1e-7
layer = Dense(10, input_dim=784, seed=1)

layer.weights = w + h
yp = layer.forward(deepcopy(X_train2))
Ep = cross_entropy(deepcopy(yp), deepcopy(y_train2))

In [13]:
# get w-
layer = Dense(10, input_dim=784, seed=1)

layer.weights = w - h
yn = layer.forward(deepcopy(X_train2))

En = cross_entropy(deepcopy(yn), deepcopy(y_train2))

In [14]:
dEdy = (Ep - En) / ( 2. * h)
# dEdW = dydW * dEdy
dEdW = dEdy
dEdW = np.dot(dEdy.T, X_train2).T

In [15]:
dEdW, dEdW.shape, w.shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), (784, 10), (784, 10))

In [16]:
# analytical dEdW
layer = Dense(10, input_dim=784, seed=1)

layer.weights = w
y_ = layer.forward(deepcopy(X_train2))
E_, dEdO_ = cross_entropy(deepcopy(y_), deepcopy(y_train2), return_deriv=True)

layer.back_propogate(deepcopy(dEdO_))
dEdW_ = layer.dEdW

In [17]:
np.sum(dEdW_)

283688923586.65063

In [18]:
dEdW_.shape

(784, 10)

In [19]:
relative_error = np.linalg.norm(dEdW - dEdW_)/ (np.linalg.norm(dEdW) + np.linalg.norm(dEdW_))

In [20]:
relative_error

0.9999983447295429

In [24]:
w

array([[ 0.65387455, -0.92132293, -0.14418936, ..., -0.39259022,
         0.01887856,  0.26064289],
       [-0.49221186,  0.51193601,  0.40320363, ..., -0.41848881,
        -0.11980319,  0.23718218],
       [-0.30932009, -0.17743357, -0.30731297, ...,  0.10483389,
         0.7422861 ,  0.33185224],
       ...,
       [ 0.48671913, -0.08406576, -0.44510272, ...,  0.00370278,
        -0.63047716, -0.40293975],
       [ 0.29035899, -0.39472819,  0.39954523, ...,  0.28816253,
        -0.68040612, -0.11703643],
       [-0.44562584, -0.48717856,  0.05977087, ...,  0.37613845,
        -0.15007702,  0.21746986]])